In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from tensorflow import keras

import DataGenerator as DG
from DataGenerator import DataGenerator

from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
#Progress bar fix: use callbacks=[Logger.JupyterProgbarLogger()] in model.fit
#verbose=0 is also required
import JupyterProgbarLogger as Logger
from tqdm import tqdm_notebook as tqdm
#from kerastuner.tuners import RandomSearch


import sklearn.metrics as metrics

import numpy as np
import random
import math

from datetime import datetime

import matplotlib.pyplot as plt

import h5py

###FIX NUMPY LOAD FOR DICTIONARIES
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [2]:
#Data Variables
BATCH_SIZE=8
DATA_AMOUNT=2000000
VALIDATION_AMOUNT=100000
#frames per sample / 3rd dimension for 3D CNN
depth=8

In [3]:
def build_model(
                input_shape=(1,80, 80, 1),
                stride_length=(1, 1, 1),
                kernel=(3,3,3),
                kernel_initializer='glorot_uniform',
                activation=layers.Activation('relu'),
                dense_activation=layers.Activation('relu'),
                output_activation=layers.Activation('softmax'),
                batch_momentum=.999,
                dropout_chance=0.2,
                combine=True,
                padding='same',
                batch_norm=False,
                dropout=False
            ):
    name = "3D CNN"
    inputs = layers.Input(shape=input_shape)
    x = inputs
    conv_parameters = {
        'padding': padding,
        'strides': stride_length,
        'kernel_initializer': kernel_initializer,
    }
    # encode net
    #if batch_norm:
    #    x = layers.BatchNormalization(momentum=batch_momentum)(x)
    #elif dropout:
    #    x = layers.Dropout(dropout_chance)(x)
    #x = activation(x)
    x = layers.Conv3D(8, kernel, **conv_parameters)(x)
    x = layers.MaxPooling3D((1, 2, 2), padding=padding)(x)
    x = layers.Conv3D(16, kernel, **conv_parameters)(x)
    x = layers.Conv3D(16, kernel, **conv_parameters)(x)
    x = layers.MaxPooling3D((2, 2, 2), padding=padding)(x)
    x = layers.Conv3D(32, kernel, **conv_parameters)(x)
    x = layers.Conv3D(32, kernel, **conv_parameters)(x)
    x = layers.MaxPooling3D((2, 2, 2), padding=padding)(x)
    x = layers.Conv3D(64, kernel, **conv_parameters)(x)
    x = layers.Conv3D(64, kernel, **conv_parameters)(x)
    x = layers.MaxPooling3D((2, 2, 2), padding=padding)(x)
    x = activation(x)
    x = layers.GlobalAveragePooling3D()(x)
    output = output_activation(x)
    model = keras.models.Model(inputs, output)
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=1e-3,
            momentum=0.9,
            nesterov=True,
            decay=1e-6
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model,name

In [4]:
if not depth == 1:
    model,name = build_model(input_shape=(depth,80,80,1))
else:
    model,name = build_model()
#print(model.summary())

W0724 14:10:36.664742 139783261407040 deprecation.py:506] From /home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
data_gen = DataGenerator("images_raw_doric_round1.h5",data_amount=DATA_AMOUNT,batch_size=BATCH_SIZE,frames_per_sample=depth)
validation_gen = DataGenerator("images_raw_doric_round1.h5",data_amount=VALIDATION_AMOUNT,batch_size=BATCH_SIZE,frames_per_sample=depth,offset=DATA_AMOUNT)
history = model.fit_generator(generator=data_gen,
                    validation_data=validation_gen,
                    epochs=25,
                    verbose=0,
                    use_multiprocessing=True,
                    workers=10,
                    callbacks=[Logger.JupyterProgbarLogger(count_mode='steps'),
                              keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=10,verbose=1, mode='auto',restore_best_weights=True)]
                   )

Epoch 1/25


Process Keras_worker_ForkPoolWorker-10:
Process Keras_worker_ForkPoolWorker-9:
Process Keras_worker_ForkPoolWorker-7:
Process Keras_worker_ForkPoolWorker-5:
Process Keras_worker_ForkPoolWorker-1:
Process Keras_worker_ForkPoolWorker-4:
Process Keras_worker_ForkPoolWorker-3:
Process Keras_worker_ForkPoolWorker-8:
Process Keras_worker_ForkPoolWorker-6:
Process Keras_worker_ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recen

  File "/home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
  File "/home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/ap487/miniconda3/envs/tf2_gpu/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/ap487/miniconda3/envs/tf2_gpu/lib

KeyboardInterrupt: 

In [ ]:
def accuracy(predictions, answers, top_n=5):
    accs = np.zeros(len(answers))
    predicts = np.zeros(len(test_labels))
    for i in range(0,len(answers)):
        j = len(predictions[i])-1
        for k in range(0,top_n):
            if predictions[i][j-k]==answers[i]:
                accs[i]=1
                predicts[i]=predictions[i][j-k]
    return np.mean(accs), predicts

In [10]:
with h5py.File("images_raw_doric_round1.h5",'r') as f:
    test_data=np.zeros((1000,depth,80,80))
    test_labels=np.zeros(1000)
    i=0
    #Starting index for batch samples
    index = 2000000
    while i < 1000:
        #Loop forward to add to the frame sequence
        cur_amount = 0
        while cur_amount < depth:
            label = f["/labels"][index]
            if label >= 0:
                test_data[i][cur_amount]=f["/frames/raw"][index]
                test_labels[i]=f["/labels"][index]
                index+=1
                cur_amount+=1
            else:
                index+=1
        i+=1
    test_data = test_data[...,None]

In [35]:
p = model.predict(test_data)
for i in range(0,len(p)):
    p[i]=np.argsort(p[i])

In [36]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
mean, predicts = accuracy(p,test_labels,top_n=2)
print(mean)
print(metrics.f1_score(test_labels,predicts,average=None))
np.set_printoptions(threshold=np.inf)
conf_mat=metrics.confusion_matrix(test_labels,[x[(len(x)-1)] for x in p])
#conf_mat=conf_mat / conf_mat.astype(np.float).sum(axis=1)
print(np.array2string(conf_mat, max_line_width=np.inf))

0.628
[0.10679612 0.81081081 0.36363636 0.7027027  0.68656716 0.5
 0.57142857 0.83870968 0.71232877 0.61538462 0.8372093  0.86666667
 0.77777778 1.         0.85       1.         0.77777778 0.76923077
 0.875      0.63636364 0.82352941 0.8        0.81481481 0.82352941
 0.91666667 0.7755102  0.44444444 1.         0.97247706 0.75
 0.         0.46511628 0.36363636 0.8        0.66666667 0.8
 0.9375     0.61538462 1.         0.81481481 1.         0.61538462
 0.         1.         0.89473684]
[[15  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  2  0  0  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0 13  2  0  0  0  0  0  0  1  1  0  0  0  0  0  2  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  3  1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  0  9  0  0  0  2  0  0  0  0  1  0  0  0  3  0  0  1  0  0  0  1  0  1  0  1  0  0  1  

In [22]:
plt.imshow(conf_mat)

NameError: name 'conf_mat' is not defined

In [44]:
filename = "model-"+datetime.today().strftime('%Y-%m-%d-%H:%M:%S')+".h5"
serialized = model.to_json()
with open(filename, "w") as f:
    f.write(serialized)
model.save_weights(filename)
print("Model saved to ",filename)

Model saved to  model-2019-07-24-13:00:01.h5


In [ ]:
model_benchmarks = np.load("benchmarks.npy")
model_benchmarks.item()[name]=metrics.f1_score(test_labels,predicts,average=None)
np.save("benchmarks.npy",model_benchmarks)
print(model_benchmarks)

In [ ]:
plt.plot(predicts)
plt.plot(test_labels)
#with h5py.File("images_raw_doric_round1.h5",'r') as f:
#    lbls = f["/labels"][:2000000:depth]
#plt.plot(lbls)
plt.xlim(800,830)
plt.show()

In [ ]:
#Old Data Function -- using DataGenerator now.
def sort_data(data,labels,train_len,val_len,test_len,blocksize=1):
    curBlock = labels[0]
    train_data = np.zeros((train_len,80,80,1))
    val_data = np.zeros((val_len,80,80,1))
    test_data = np.zeros((test_len,80,80,1))
    train_labels = np.full(train_len,-1)
    val_labels = np.full(val_len,-1)
    test_labels = np.full(test_len,-1)
    train_ind = 0
    val_ind = 0
    test_ind = 0
    choices = np.arange(3)
    i=0
    cur_len = 0
    while i < len(data):
        start = i-cur_len
        iter_amt = len(data) if blocksize == 1 else blocksize
        while labels[i]==curBlock and cur_len < iter_amt:
            i+=1
            cur_len+=1
            if i == len(data):
                break
        end = i
        if not i == len(data):
            if labels[i]==curBlock:
                cur_len-=int(blocksize/2)
            else:
                cur_len=0
            curBlock = labels[i]
        if not blocksize == 1 and end-start < blocksize:
            continue
        cur_frames = data[start:end]
        cur_labels = labels[start:end]
        choice=-1
        if choices.size>0:
            choice = np.random.choice(choices)
        else:
            break
        if choice == 0:
            if train_ind + len(cur_frames) >= train_len:
                cur_frames = cur_frames[:train_len-train_ind]
                cur_labels = cur_labels[:train_len-train_ind]
                choices=np.delete(choices,np.argwhere(choices==0))
            train_data[train_ind:train_ind+len(cur_frames)]=cur_frames
            train_labels[train_ind:train_ind+len(cur_frames)]=cur_labels
            train_ind += len(cur_frames)
        elif choice == 1:
            if val_ind + len(cur_frames) >= val_len:
                cur_frames = cur_frames[:val_len-val_ind]
                cur_labels = cur_labels[:val_len-val_ind]
                choices=np.delete(choices,np.argwhere(choices==1))
            val_data[val_ind:val_ind+len(cur_frames)]=cur_frames
            val_labels[val_ind:val_ind+len(cur_frames)]=cur_labels
            val_ind += len(cur_frames)
        elif choice == 2:
            if test_ind + len(cur_frames) >= test_len:
                cur_frames = cur_frames[:test_len-test_ind]
                cur_labels = cur_labels[:test_len-test_ind]
                choices=np.delete(choices,np.argwhere(choices==2))
            test_data[test_ind:test_ind+len(cur_frames)]=cur_frames
            test_labels[test_ind:test_ind+len(cur_frames)]=cur_labels
            test_ind += len(cur_frames)
    return train_data[:train_ind-(train_ind%blocksize)],train_labels[:train_ind-(train_ind%blocksize)],val_data[:val_ind-(val_ind%blocksize)],val_labels[:val_ind-(val_ind%blocksize)],test_data[:test_ind-(test_ind%blocksize)],test_labels[:test_ind-(test_ind%blocksize)]

In [ ]:
with h5py.File("images_raw_doric_round1.h5",'r') as f:
    print(f["/labels"][0:130])
data_gen.__getitem__(1)[1]

In [ ]:
pl = model.predict(data_gen.__getitem__(518)[0])
plc = np.zeros(len(pl))
for i in range (0,len(pl)):
    plc[i]=np.argmax(pl[i])
plt.plot(plc)
plt.plot(data_gen.__getitem__(518)[1])
plt.show()

In [12]:
!nvidia-smi

Wed Jul 24 13:12:01 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.48                 Driver Version: 410.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:04:00.0 Off |                    0 |
| N/A   69C    P0   147W / 149W |  10943MiB / 11441MiB |     96%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:05:00.0 Off |                    0 |
| N/A   55C    P0   101W / 149W |  10941MiB / 11441MiB |     78%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [9]:
#data_gen = DataGenerator("images_raw_doric_round1.h5",data_amount=DATA_AMOUNT,batch_size=BATCH_SIZE,frames_per_sample=depth)
data_gen.__getitem__(2)

(array([[[[[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
 
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
 
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
 
          ...,
 
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
 
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
 
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]]],
 
 
         [[[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
 
          [[0.],
           [0.],
           [0.],
           ...,
           [0.],
           [0.],
           [0.]],
 
          [[